In [1]:
import os
import openml
import pandas as pd
import numpy as np
from scipy.io.arff import loadarff
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from fastprogress.fastprogress import master_bar, progress_bar

In [2]:
seed = 42

## TODO
- save dataframes with {name}\_{mean_score}\_{std_score}.csv
- initially test only on classical suite than extend to all classification datasets
- noise injection for "data augmentation"
- explore other data augmentation
- hot encoding (as being applied in openml) could be terrible option, try target encoding or other method

In [3]:
path = '../data/'

os.makedirs('../samples', exist_ok=True)

In [4]:
openml_list = openml.datasets.list_datasets()
suites = openml.study.list_suites(output_format="dataframe", status="all")

In [5]:
suites.head(100)

,id,alias,main_entity_type,name,status,creation_date,creator
14,14,OpenML100,task,"Collaborative, reproducible benchmarking and a...",in_preparation,2019-02-21 18:40:13,1
99,99,OpenML-CC18,task,OpenML Benchmarking Suites and the OpenML-CC18,in_preparation,2019-02-21 18:47:13,1
216,216,CoolStudy,task,A Cool Study with an Awesome name,in_preparation,2019-02-28 23:00:26,8006
217,217,Study,task,A Cool Study with an Awesome name,in_preparation,2019-02-28 23:05:11,8006
218,218,AutoML-Benchmark,task,AutoML Benchmark,in_preparation,2019-05-02 13:35:08,869
219,219,FOREX,task,Forex,in_preparation,2019-06-04 00:45:17,1
225,225,OpenML-friendly,task,OpenML100-friendly,active,2019-09-16 19:41:46,1
236,236,a9ee1f0b2a4b48b6b6da1653fe92890e,task,Item Response Theory for Classification problems,in_preparation,2020-04-06 21:38:55,64
239,239,c638a5d3d31241179f9b4853951fdb79,task,Item Response Theory for Regression problems,in_preparation,2020-04-19 22:15:30,64
240,240,e5e7f56c8655433eb2418c240ec8b8c0,task,InvestigatingDL,in_preparation,2020-04-28 02:30:38,2902


In [6]:
suite = openml.study.get_suite(271)
print(suite)

OpenML Benchmark Suite
ID..............: 271
Name............: AutoML Benchmark All Classification
Status..........: in_preparation
Main Entity Type: task
Study URL.......: https://www.openml.org/s/271
# of Data.......: 71
# of Tasks......: 71
Creator.........: https://www.openml.org/u/869
Upload Time.....: 2020-11-19 20:52:19


In [7]:
datasets = suite.data

In [8]:
# for idd in progress_bar(datasets[:-1]):
#     dataset = openml.datasets.get_dataset(idd)
#     print(f'Getting dataset id {idd}, {dataset.name}...')
#     X, y, categorical_indicator, attribute_names = dataset.get_data(
#         dataset_format="array", target=dataset.default_target_attribute
#     )
#     df = pd.DataFrame(X, columns=attribute_names)
#     df["class"] = y
#     df.to_csv(path+dataset.name+'.csv',index=False)

In [9]:
clf1 = RandomForestClassifier(random_state=seed)

In [11]:
mb = master_bar(os.listdir(path))

for f in mb:
    mb.main_bar.comment = f'Files'
    sampler = pd.read_csv(path+f).dropna()
    if sampler.shape[0] < 1000:
        continue
    for i in progress_bar(range(10), parent=mb):
        df = sampler.sample(500, replace=True, random_state=seed+i)
        X = df.drop(["class"], axis=1).astype(str)
        y = df["class"].astype(str)
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        scores = []
        for train_idx, test_idx in kfold.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
            clf1.fit(X_train, y_train)
            scores.append(clf1.score(X_test, y_test))
        df.to_csv('../samples/'+f'{f}_{np.mean(scores):.3f}_{i}.csv', index=False)
        mb.child.comment = f'Sampler'
    mb.write(f'Finished {f}')

/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)

/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)

/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jader/Projects/deep-meta-learning/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)